In [ ]:
import os
import json
import uuid
import time
import logging
from typing import List, Annotated
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_groq.chat_models import ChatGroq
from pydantic import BaseModel
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.agent_toolkits import FileManagementToolkit

load_dotenv()

In [ ]:
# --- File Management Toolkit ---
# Using the current directory for file operations.
# For a real application, you might want to specify a dedicated directory.
file_toolkit = FileManagementToolkit(
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()

read_file, write_file, list_directory = file_toolkit

In [ ]:
# --- Logging Setup ---
def setup_logger(name="srs_agent_logger", log_file="srs_agent.log", level=logging.INFO):
    """Sets up a logger with both file and console output."""
    logger = logging.getLogger(name)
    logger.setLevel(level)

    if not logger.handlers:
        # Formatter
        formatter = logging.Formatter(
            "[%(asctime)s] [%(levelname)s] %(name)s: %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S"
        )

        # Console Handler
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)

        # File Handler
        log_dir = os.path.dirname(log_file)
        if log_dir:  # Only create folder if path exists
            os.makedirs(log_dir, exist_ok=True)
        file_handler = logging.FileHandler(log_file)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

    return logger

logger = setup_logger()

In [ ]:
# --- Pydantic Models for Structured Output ---
class Information(BaseModel):
    """Structured information gathered from the user for the SRS."""
    drivers: str
    personas: str
    functionalities: str
    constraints: str
    other: str

In [ ]:
INFO_GATHER_PROMPT = """
Your role is to act as a user-centric requirements elicitation assistant.  
You will gather precise, actionable details from the user about the software or application they want to create.  
Your objective is to collect information that is specific enough to define use cases and design requirements without making unfounded assumptions.

You must obtain the following information:

1. Personas – Identify all intended user types:
   - Who will use the software/application?
   - Their roles, responsibilities, and level of technical expertise.
   - Context of use (environment, workflow, frequency).

2. Functionalities – For each persona:
   - What do they want to accomplish?
   - Key features, workflows, and interactions.
   - Expected inputs/outputs.

3. Drivers / Business Goals – Why this software is needed:
   - Example: cost optimization, performance, scalability, availability, reliability, security, compliance, operational efficiency, innovation, etc.

4. Constraints – Any limitations:
   - Timeframe, budget, technology stack, skills, regulations, integrations, or infrastructure constraints.

5. Other Details – Any additional context:
   - Industry-specific requirements, preferred tools, existing pain points, data sensitivity, future scalability considerations.

Interaction Rules:
- Do not guess or fabricate details unless explicitly instructed by the user to suggest options.  
- If any required area is unclear or incomplete, ask targeted follow-up questions to fill gaps.  
- When users struggle to answer, provide examples or structured choices to make answering easier.  
- After each response, confirm your understanding by restating the information in concise bullet points and asking for validation.  
- Only proceed to call the relevant tool once Personas, Functionalities, and Constraints are fully understood and validated.
"""

In [ ]:
BA_PROMPT = """ 
-- ROLE --
You are acting as an experienced Business Analyst responsible for producing a complete, accurate, and unambiguous Software Requirements Specification (SRS).

-- OBJECTIVE --
Using the provided user clarifications and requirements, your task is to create a professional SRS document that is ready for stakeholder review and developer handoff.

-- CRITICAL EXECUTION RULES --
1. You MUST fully review and understand all provided inputs before writing the SRS.  
2. Follow the Analysis process step-by-step without skipping or merging steps.  
3. Structure and content of the output MUST exactly match the provided Expected Output template.

-- INPUTS --
You will receive the following:
1. Clarifications / Requirements from the User: {reqs}

-- ANALYSIS PROCESS --
1. Interpretation – Parse the provided inputs, ensuring that all ambiguities, missing details, or contradictions are resolved.  
2. SRS Drafting – Develop the SRS strictly from the validated inputs, avoiding assumptions unless explicitly authorized by the user.  
3. Template Compliance – Format the document precisely according to the given Expected Output structure.

-- EXPECTED OUTPUT --
The output MUST:
- Use the following format: {format}
- Present information with clear headings, numbering, and consistent terminology.
- Ensure all functional, non-functional, and constraint-related requirements are explicitly documented.
- Avoid vague terms such as “etc.” or “and so on” — every requirement should be specific and measurable.


"""

In [ ]:
BA_REVIEW_PROMPT = """ 
-- ROLE --
You are acting as an experienced Business Analyst responsible for maintaining and improving a Software Requirements Specification (SRS).

-- OBJECTIVE --
Your task is to produce an dated SRShat fully incorporates the provided review feedback while preserving all existing valid content.

-- CRITICAL EXECUTION RULES --
1. Review puts in their entirety before making changes.  
2. Follow the Analysis process exactly as listed — no skipping or combining steps.  
3. All review comments must be addressed explicitly; no comment should be ignored or partially applied.  
4. Preserve all original, relevant content unless the feedback specifies removal or replacement.  
5. The updated SRS MUST strictly follow the provided Expected Output format.

-- INPUTS --
You will receive:
1. Current Version of the SRS (full text).
2. Review Comments from Reviewer: {review}

-- ANALYSIS PROCESS --
1. Comprehension – Read and fully understand the current SRS and the review comments.  
2. Mapping Identify exactly where each review comment applies in the SRS.  
3. Application – Apply updates to the SRS while ensuring:
   - All feedback is incorporated in the relevant sections.
   - No unrelated or correct information is altered or removed.
4. Compliance Check Verify that:
   - All comments have been addressed.
   - The document structure matches the cted Output format.
   - No original requirement is lost or unintentionally modified.

-- EXPECTED OUTPUT --
Produce the Updated SRS in the following format: {format}  
Ensure:
- All sections are clearly labeled and numbered.
- Formatting, terminology, and style remain consistent.
- Changes are seamlessly integrated so the document reads as a single, cohesive version.

"""

In [ ]:
CRITIQUE_PROMPT = """ 
-- ROLE --
You are acting as a nior Business Analystked solely with reviewing the provided ware Requirements Specification (SRS)

-- OBJECTIVE --
Provide a concise, high-value review highlighting only meaningful improvement points.  
You MUST NOT make direct changes to the SRS content.

-- CRITICAL EXECUTION RULES --
1. Review the putsn full before producing feedback.  
2. Follow the Reviewrocess exactly — no skipping or adding unrelated steps.  
3. Do not rewrite or edit the SRS; only evaluate and comment.  
4. Avoid trivial or obvious remarks; focus only on critical observations that impact clarity, completeness, or professionalism.

-- INPUTS --
1. Software Requirements Specificationfull text).

-- REVIEW PROCESS --
1. Assess whether the SRS is:
   - Formal in tone and style.
   - Professionally structured and organized.
   - Clear, complete, and free from ambiguity.
2. Identify only significant issues (critical gaps, inconsistencies, missing details, unclear terminology, structural problems).
3. Keep review comments short, concise, and in bullet points
4. Exclude any unnecessary praise, minor formatting suggestions, or restatements of what is already correct.

-- DESIRED OUTPUT --
`ReqReview = <Say "Satisfied" if there are no review comments; otherwise say "Enhance" followed by a bullet-point list of critical review comments. Include "Alignment Scope: <in %>" where percentage reflects how much of the SRS already meets professional standards>`  

"""

In [ ]:
# --- Graph State ---
class State(TypedDict):
    messages: Annotated[list, add_messages]
    srs: Annotated[list, add_messages]
    srs_format: Annotated[list, add_messages]
    max_iteration: int
    iteration: int

In [ ]:
# --- Agent Nodes ---
llm = ChatGroq(model='moonshotai/kimi-k2-instruct', temperature=0.2, max_retries=2)
llm_with_tool = llm.bind_tools([Information])

def information_gathering(state: State):
    """Gathers initial requirements from the user."""
    logger.info("--- INFORMATION GATHERING ---")
    messages = [SystemMessage(content=INFO_GATHER_PROMPT)] + state["messages"]
    
    if state.get('srs'):
        return {"messages": [AIMessage(content="Proceeding to the next step.")]}
    
    response = llm_with_tool.invoke(messages)
    return {"messages": [response]}

def conclude_conversation(state: State):
    """Concludes the information gathering phase."""
    logger.info("--- CONCLUDING CONVERSATION ---")
    return {
        "messages": [
            ToolMessage(
                content="Information gathering complete. Proceeding to generate SRS.",
                tool_call_id=state["messages"][-1].tool_calls[0]["id"],
            )
        ]
    }

def get_ba_prompt_messages(state: State):
    """Prepares messages for the Business Analyst agent."""
    tool_call = None
    other_msgs = []
    messages = state["messages"]
    srs_format = state["srs_format"][-1].content
    
    for m in messages:
        if isinstance(m, AIMessage) and m.tool_calls:
            tool_call = m.tool_calls[0]["args"]
        elif isinstance(m, ToolMessage):
            continue
        elif tool_call is not None:
            other_msgs.append(m)

    iteration = state['iteration']
    last_message = state['messages'][-1].content
    logger.info(f"--- SRS Generation/Revision Number: {iteration} ---")
        
    srs = state.get('srs')
    if srs:
        current_srs = state['srs'][-1].content
        return [
            SystemMessage(content=BA_REVIEW_PROMPT.format(review=last_message, format=srs_format)),
            HumanMessage(content=current_srs)
        ]
    else:
        return [
            SystemMessage(content=BA_PROMPT.format(reqs=tool_call, format=srs_format)),
        ] + other_msgs

def generate_srs(state: State):
    """Generates or revises the SRS document."""
    logger.info("--- GENERATING/REVISING SRS ---")
    messages = get_ba_prompt_messages(state)
    response = llm.invoke(messages)

    iteration = state['iteration']
    file_name = f"output/srs_v{iteration}.md"
    
    write_file.invoke({"file_path": file_name, "text": response.content})
    logger.info(f"SRS saved to {file_name}")
    
    return {
        "messages": [response],
        "srs": [response]
    }

def reviewer(state: State):
    """Reviews the generated SRS and provides feedback."""
    logger.info("--- REVIEWING SRS ---")
    srs = state['srs']
    messages = [
        SystemMessage(content=CRITIQUE_PROMPT),
        HumanMessage(content=srs[-1].content)
    ]
    response = llm.invoke(messages)
    
    iteration = state['iteration'] + 1
    file_name = f"output/srs_feedback_v{iteration}.md"
    
    write_file.invoke({"file_path": file_name, "text": response.content})
    logger.info(f"Review feedback saved to {file_name}")
    
    return {
        "messages": [response],
        "iteration": iteration
    }

# --- Conditional Edges ---
def is_clarified(state: State):
    """Checks if the information gathering is complete."""
    messages = state["messages"]
    if isinstance(messages[-1], AIMessage) and messages[-1].tool_calls:
        return "yes"
    return "no"

def is_reviewed(state: State):
    """Checks if the review process is complete."""
    max_iteration = state['max_iteration']
    iteration = state['iteration']
    last_message = state['messages'][-1].content
   
    if "satisfied" in last_message.lower():
        logger.info("--- REVIEW SATISFIED ---")
        return 'reviewed'
    elif iteration > max_iteration:
        logger.info("--- MAX ITERATIONS REACHED ---")
        return 'reviewed'
    else:
        logger.info("--- ENHANCEMENT REQUIRED ---")
        return 'enhance'

In [ ]:
# --- Graph Definition ---
memory = MemorySaver()
workflow = StateGraph(State)

workflow.add_node("information_gathering", information_gathering)
workflow.add_node("generate_srs", generate_srs)
workflow.add_node("conclude_conversation", conclude_conversation)
workflow.add_node("reviewer", reviewer)

workflow.add_edge(START, "information_gathering")

workflow.add_conditional_edges(
    "information_gathering", 
    is_clarified, 
    {"yes": "conclude_conversation", "no": END}
)

workflow.add_conditional_edges(
    "reviewer", 
    is_reviewed, 
    {"reviewed": END, "enhance": "generate_srs"}
)

workflow.add_edge("conclude_conversation", "generate_srs")
workflow.add_edge("generate_srs", "reviewer")

graph = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
if __name__ == "__main__":
    thread = {"configurable": {"thread_id": str(uuid.uuid4())}}
    try:
        srs_format_content = read_file.invoke({"file_path": "input/srs_format.md"})
    except Exception as e:
        logger.error(f"Error reading srs_format.md: {e}")
        srs_format_content = "Please generate a standard SRS document."

    while True:
        user_input = input("User (q/Q to quit): ")
        if user_input.lower() in ["quit", "q"]:
            print("AI: Goodbye!")
            break
        
        events = graph.stream(
            {
                "messages": [HumanMessage(content=user_input)],
                "srs_format": [HumanMessage(content=srs_format_content)],
                "iteration": 1,
                "max_iteration": 3,
            }, 
            config=thread, 
            stream_mode="values"
        )
        
        for event in events:
            if "messages" in event:
                event["messages"][-1].pretty_print()